## Imports

In [1]:
from constants import FLUXES_FOLDER, SLM_FOLDER

from data_utils import load_numpy_data, \
                       flatten_data, \
                       shuffle_arrays, \
                       fuse_amplitude_and_phase, \
                       split_data, \
                       normalize_data

from plot_utils import plot_map, \
                       plot_model_history

from modeling_utils import create_fully_connected_architecture_for_amplitude_and_phase_reconstruction, \
                           compile_linear_model_for_amplitude_reconstruction, \
                           train_linear_model_for_amplitude_reconstruction

2023-11-27 10:34:17.702108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 10:34:18.337334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load data

In the first place, define the data paths

In [2]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"
phases_filename = "complexsine_pupphase.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"
phases_path = f"{SLM_FOLDER}/{phases_filename}"

Read the numpy raw files

In [3]:
n_points = 10000
fluxes_array = load_numpy_data(fluxes_path,
                               n_points)

amplitudes_array = load_numpy_data(amplitudes_path,
                                   n_points)

phases_array = load_numpy_data(phases_path,
                               n_points)

amplitudes_array, amplitudes_scaler = normalize_data(amplitudes_array)
phases_array, phases_scaler = normalize_data(phases_array)

Normalize data

In [4]:
fluxes_array, fluxes_scaler = normalize_data(fluxes_array)
amplitudes_array, amplitudes_scaler = normalize_data(amplitudes_array)
phases_array, phases_scaler = normalize_data(phases_array)

Shuffle the data

In [5]:
fluxes_array, amplitudes_array, phases_array = shuffle_arrays([fluxes_array,
                                                               amplitudes_array,
                                                               phases_array])

Merge phase and amplitude

In [6]:
amp_phase_array = fuse_amplitude_and_phase(amplitudes_array,
                                           phases_array)

Flatten input fluxes

In [9]:
flattened_fluxes_array = flatten_data(fluxes_array)
flattened_fluxes_array.shape

(10000, 1320)

Split data in train and validation set

In [10]:
val_ratio = 0.1
train_flattened_fluxes_array, val_flattened_fluxes_array = split_data(flattened_fluxes_array,
                                                                      val_ratio)

train_amp_phase_array, val_amp_phase_array = split_data(amp_phase_array,
                                                        val_ratio)

In [11]:
amp_phase_array.shape

(10000, 2, 96, 96)

In [12]:
train_amp_phase_array.shape

(9000, 2, 96, 96)

In [13]:
train_flattened_fluxes_array.shape

(9000, 1320)

## Define the model

In [14]:
from tensorflow import keras

input_shape = train_flattened_fluxes_array[0].shape
output_shape = train_amp_phase_array[0].shape
hidden_layer_sizes = [1024, 2048, 2048]
regularizer = keras.regularizers.L1L2(l1=0.001,l2=0.1)
hidden_activation = 'relu'
output_activation = 'linear'
use_batch_normalization = True
model_name = "AmplitudePhaseReconstructor1"

In [15]:
model = create_fully_connected_architecture_for_amplitude_and_phase_reconstruction(
    input_shape,
    output_shape,
    hidden_layer_sizes,
    regularizer,
    hidden_activation,
    output_activation,
    use_batch_normalization,
    model_name
)

18432
18432


2023-11-27 10:34:47.265624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 10:34:47.295190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 10:34:47.295469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Compile model

In [16]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

loss_function = tf.keras.losses.MeanSquaredError()
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
metric = tf.keras.metrics.MeanSquaredError()

compile_linear_model_for_amplitude_reconstruction(
    model,
    loss_function,
    optimizer,
    metric
)

## Train model

In [17]:
epochs = 50
batch_size = 128
reduce_lr = ReduceLROnPlateau('mean_squared_error', factor=0.1, patience=15, verbose=1)
early_stop = EarlyStopping('mean_squared_error', patience=50, verbose=1)
callbacks = [reduce_lr, early_stop]

history = train_linear_model_for_amplitude_reconstruction(
    model,
    train_flattened_fluxes_array,
    train_amp_phase_array,
    batch_size,
    epochs,
    val_flattened_fluxes_array,
    val_amp_phase_array,
    callbacks
)

2023-11-27 10:34:48.412157: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 663552000 exceeds 10% of free system memory.
2023-11-27 10:34:48.927693: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 663552000 exceeds 10% of free system memory.


Epoch 1/50


2023-11-27 10:34:51.122488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-27 10:34:51.130054: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1d31ae10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-27 10:34:51.130092: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-11-27 10:34:51.142403: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-27 10:34:51.344245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-27 10:34:51.454365: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime o

71/71 [==============================] - 5s 25ms/step - loss: 378.4664 - mean_squared_error: 0.8940 - val_loss: 38.8541 - val_mean_squared_error: 0.9393 - lr: 0.0010
Epoch 2/50
71/71 [==============================] - 1s 21ms/step - loss: 11.4488 - mean_squared_error: 0.9043 - val_loss: 3.5385 - val_mean_squared_error: 0.9624 - lr: 0.0010
Epoch 3/50
71/71 [==============================] - 1s 21ms/step - loss: 2.7731 - mean_squared_error: 0.8971 - val_loss: 1.8128 - val_mean_squared_error: 0.9353 - lr: 0.0010
Epoch 4/50
71/71 [==============================] - 2s 21ms/step - loss: 1.4752 - mean_squared_error: 0.8390 - val_loss: 1.4521 - val_mean_squared_error: 0.8932 - lr: 0.0010
Epoch 5/50
71/71 [==============================] - 1s 21ms/step - loss: 1.4139 - mean_squared_error: 0.8334 - val_loss: 1.4618 - val_mean_squared_error: 0.8602 - lr: 0.0010
Epoch 6/50
71/71 [==============================] - 1s 21ms/step - loss: 4.6276 - mean_squared_error: 0.9858 - val_loss: 10.2891 - val_me

Epoch 47/50
71/71 [==============================] - 1s 21ms/step - loss: 0.9116 - mean_squared_error: 0.8298 - val_loss: 0.9220 - val_mean_squared_error: 0.8301 - lr: 1.0000e-04
Epoch 48/50
71/71 [==============================] - 1s 21ms/step - loss: 0.9118 - mean_squared_error: 0.8297 - val_loss: 0.9280 - val_mean_squared_error: 0.8301 - lr: 1.0000e-04
Epoch 49/50
71/71 [==============================] - 1s 21ms/step - loss: 0.9124 - mean_squared_error: 0.8297 - val_loss: 0.9231 - val_mean_squared_error: 0.8301 - lr: 1.0000e-04
Epoch 50/50
71/71 [==============================] - 1s 20ms/step - loss: 0.9128 - mean_squared_error: 0.8298 - val_loss: 0.9022 - val_mean_squared_error: 0.8301 - lr: 1.0000e-04


In [21]:
n = 10
plot_map(amplitudes_array[n])
plot_map(phases_array[n])
im = model.predict(train_flattened_fluxes_array[n].reshape(1,1320))
plot_map(im[0][0])
plot_map(im[0][1])
plot_map(train_flattened_fluxes_array[n].reshape(fluxes_array[n].shape))

1/1 [==============================] - 0s 15ms/step
